In [2]:
%load_ext autoreload
%autoreload 2
import os
from huggingface_hub import login
from collections import defaultdict
from mttl.models.library.utils import get_svd_embedding

from mttl.models.library.expert_library import LocalExpertLibrary,  HFExpertLibrary
from mttl.models.library.library_transforms import SVDEmbeddingTransform, SVDEmbeddingTransformConfig
from huggingface_hub import login, HfApi
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

/anaconda/envs/comp_3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/anaconda/envs/comp_3.9/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warning(


Registering modifier...lora
Registering modifier...poly
Registering modifier...per_token_poly
Registering modifier...skilled
Registering modifier...kv_adapter
Registering modifier...poly_kv_adapter
Registering modifier...prompt_tuning
Registering modifier...poly_prompt_tuning
Registering modifier...hard_prompt
Registering multi-expert selector...poly_router
Registering multi-expert selector...moe_rkhs_router
Registering multi-expert selector...zero_router
Registering multi-expert selector...zero_per_token_router
Registering multi-expert selector...poly_router_dir
Registering multi-expert selector...uniform
Registering multi-expert selector...info_selector
Registering multi-expert selector...task_selector
Registering multi-expert selector...kv_task_selector
Registering multi-expert selector...kv_concat_selector
Registering multi-expert selector...kv_norm_selector
Registering multi-expert selector...kv_concat_norm_selector
Registering multi-expert selector...kv_task_norm_selector


## Create clsuters

In [31]:
hf_api_key = os.environ["HF_TOKEN"]
login(token=hf_api_key)
user = HfApi(token=hf_api_key).whoami()
# hf_repo_id = "ostapeno/library-gptneo_1B_flan_2ep"
# hf_repo_id = "ostapeno/library-gptneo_1B_flan_2ep_underparam"
# hf_repo_id = "ostapeno/library-stablelm_flan_5ep"
hf_repo_id = "sordonia/library-phi_2-v3-2epc"

local_lib_location = f"/tmp/{hf_repo_id}"
if not os.path.exists(local_lib_location):
    os.makedirs(local_lib_location)
    expert_lib: LocalExpertLibrary = LocalExpertLibrary.from_expert_library(
        HFExpertLibrary(hf_repo_id), local_lib_location
    )
else:
    expert_lib: LocalExpertLibrary = LocalExpertLibrary(local_lib_location)

# import json
# task_set_path = "/home/v-oostapenko/dev/lucas_mttl/projects/wiki_experts/task_sets/flan_tasks.json"
# flan256 = json.load(open(task_set_path))["flan256"]

# for expert_name, expert in list(expert_lib.data.items()):
#     if expert.expert_task_name not in flan256:
#         expert_lib.remove_expert(expert_name)
# assert len(expert_lib) == 256

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/v-oostapenko/.cache/huggingface/token
Login successful


In [32]:
for name, exp in expert_lib.data.items():
    # print task
    print(f"\"{exp.expert_task_name}\"")

"glue_sst2_2_0_0"
"dream_read_the_following_conversation_and_answer_the_question"
"race_middle_Read_the_article_and_answer_the_question_no_option_"
"adversarial_qa_droberta_generate_question"
"adversarial_qa_dbidaf_question_context_answer"
"app_reviews_convert_to_star_rating"
"race_high_Select_the_best_answer"
"super_glue_rte_1_0_2"
"true_case"
"wiqa_what_might_be_the_first_step_of_the_process"
"quail_description_context_question_answer_id"
"quail_context_question_description_text"
"stream_qed"
"huggingface_xsum"
"cos_e_v1_11_question_option_description_text"
"wiqa_what_is_the_final_step_of_the_following_process"
"ropes_background_new_situation_answer"
"wiki_qa_found_on_google"
"cot_esnli"
"social_i_qa_Show_choices_and_generate_answer"
"cot_gsm8k"
"app_reviews_categorize_rating_using_review"
"cot_sensemaking"
"trec_1_0_0"
"super_glue_wic_1_0_2"
"ropes_prompt_bottom_no_hint"
"quartz_answer_question_based_on"
"super_glue_record_1_0_2"
"yelp_polarity_reviews_0_2_0"
"race_middle_Is_this_th

In [33]:
def create_embeddings():
    svd_embedder = SVDEmbeddingTransform(
        SVDEmbeddingTransformConfig(sparsity_threshold=0.1),
        random_state=42,
    )
    embeddings, svd = svd_embedder.transform(expert_lib, persist=True, force=True)
    del svd_embedder
    return embeddings, svd


embeds = expert_lib.get_auxiliary_data("embeddings")
# if len(embeds) == 0:
print("creating embeddings")
_, svd = create_embeddings()

# module to embedding
module2embed = {}
for n, m in expert_lib.items():
    module2embed[n] = get_svd_embedding(expert_lib, n)

creating embeddings


100%|██████████| 256/256 [00:04<00:00, 53.89it/s]


In [5]:
# Extract the embeddings as a numpy array
embeddings = np.array(list(module2embed.values()))
cosine_sim_matrix = cosine_similarity(embeddings, embeddings)
K = 5
kmeans = KMeans(n_clusters=K, init="k-means++", n_init=10, random_state=42)
kmeans.fit(cosine_sim_matrix)
cluster_labels = kmeans.labels_

In [1]:
clusters = defaultdict(list)
# Print the cluster labels for each embedding
for key, label in zip(module2embed.keys(), cluster_labels):
    clusters[label].append(key)

for c, l in clusters.items():
    # print(f"Cluster {c} has {len(l)} elements")
    print(f"c{c}o{K}_2e = {l}")

NameError: name 'defaultdict' is not defined

## Balanced k-means


In [34]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from kmeans_pytorch import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import datasets
import numpy as np
from tqdm.auto import tqdm


In [35]:
seed = 42
num_clusters = 10
balanced = True
debug = True
# set random seed
np.random.seed(seed)

In [47]:
device = torch.device('cuda:0')
embeddings = np.array(list(module2embed.values()))
cosine_sim_matrix = cosine_similarity(embeddings, embeddings)
X = torch.from_numpy(embeddings) #torch.from_numpy(cosine_sim_matrix).to(device)
kmeans = KMeans(n_clusters=num_clusters, device=device, balanced=balanced)

In [48]:
cluster_ids = kmeans.fit(
    X=X, distance='cosine', iter_limit=100, tqdm_flag=True, online=False
)

running k-means on cuda:0..


[running kmeans]: 0it [00:00, ?it/s]

In [49]:
cluster_ids

tensor([7, 4, 9, 2, 6, 1, 9, 8, 7, 5, 9, 9, 0, 0, 4, 2, 3, 8, 8, 8, 1, 8, 4, 7,
        7, 3, 5, 0, 7, 9, 3, 5, 2, 3, 2, 9, 8, 6, 3, 6, 5, 5, 0, 5, 0, 5, 6, 7,
        0, 2, 2, 7, 9, 9, 7, 7, 4, 2, 1, 5, 1, 9, 5, 0, 3, 7, 5, 6, 4, 1, 3, 8,
        1, 6, 8, 7, 2, 5, 2, 6, 1, 1, 0, 1, 5, 2, 3, 2, 4, 5, 0, 6, 9, 2, 0, 2,
        8, 1, 1, 9, 5, 0, 0, 4, 3, 1, 6, 1, 3, 1, 9, 9, 8, 3, 9, 4, 3, 0, 7, 7,
        5, 6, 7, 9, 9, 0, 4, 6, 5, 8, 6, 2, 1, 6, 4, 3, 7, 3, 9, 6, 2, 5, 9, 4,
        3, 1, 1, 1, 6, 4, 8, 3, 6, 6, 5, 7, 8, 8, 4, 4, 4, 2, 0, 0, 8, 3, 0, 2,
        2, 7, 4, 1, 3, 4, 0, 1, 8, 9, 4, 9, 1, 2, 7, 5, 0, 0, 7, 3, 0, 8, 3, 4,
        6, 8, 9, 7, 8, 2, 4, 2, 1, 7, 6, 4, 0, 9, 8, 2, 3, 0, 5, 3, 8, 7, 4, 1,
        4, 6, 2, 0, 8, 9, 7, 8, 2, 6, 6, 2, 1, 6, 5, 5, 7, 9, 5, 0, 9, 8, 0, 7,
        4, 6, 3, 0, 5, 6, 3, 8, 0, 4, 0, 1, 0, 3, 5, 0])

In [52]:
clusters = defaultdict(list)
# Print the cluster labels for each embedding
for key, label in zip(module2embed.keys(), list(cluster_ids)):
    clusters[label.item()].append(key)

for c, l in clusters.items():
    # print(f"Cluster {c} has {len(l)} elements")
    print(f"c{c}o{num_clusters}_2e = {l}")

c7o10_2e = ['glue_sst2_2_0_0', 'true_case', 'trec_1_0_0', 'super_glue_wic_1_0_2', 'yelp_polarity_reviews_0_2_0', 'wmt16_translate_ro_en_1_0_0', 'glue_cola_2_0_0', 'ag_news_subset_1_0_0', 'paws_wiki_1_1_0', 'super_glue_wsc_fixed_1_0_2', 'math_dataset_algebra__linear_1d_1_0_0', 'glue_qqp_2_0_0', 'fix_punct', 'imdb_reviews_plain_text_1_0_0', 'gigaword_1_2_0', 'word_segment', 'wmt16_translate_de_en_1_0_0', 'glue_stsb_2_0_0', 'wmt16_translate_fi_en_1_0_0', 'super_glue_copa_1_0_2', 'para_crawl_enes', 'wmt14_translate_fr_en_1_0_0', 'wmt16_translate_tr_en_1_0_0', 'definite_pronoun_resolution_1_1_0', 'glue_mrpc_2_0_0']
c4o10_2e = ['dream_read_the_following_conversation_and_answer_the_question', 'cos_e_v1_11_question_option_description_text', 'cot_sensemaking', 'sciq_Multiple_Choice', 'cos_e_v1_11_question_description_option_text', 'qasc_qa_with_separated_facts_2', 'cos_e_v1_11_question_option_description_id', 'cot_ecqa_ii', 'quarel_do_not_use', 'dream_baseline', 'cos_e_v1_11_question_descriptio

### Rand score

In [14]:
from sklearn.metrics.cluster import adjusted_rand_score

In [29]:
adjusted_rand_score(cluster_labels, cluster_labels_direct)

0.47611902409439244

# Save svd file

In [ ]:
hf_api_key = os.environ["HF_TOKEN"]
login(token=hf_api_key)
user = HfApi(token=hf_api_key).whoami()
hf_repo_id = "ostapeno/library-phi_2-v3-10-flan-clusters"
local_lib_location = f"/tmp/{hf_repo_id}"
if not os.path.exists(local_lib_location):
    os.makedirs(local_lib_location)
    expert_lib: LocalExpertLibrary = LocalExpertLibrary.from_expert_library(
        HFExpertLibrary(hf_repo_id), local_lib_location
    )
else:
    expert_lib: LocalExpertLibrary = LocalExpertLibrary(local_lib_location)

In [ ]:
_, svd = create_embeddings()

In [ ]:
# upload embeddings
remote_lib = HFExpertLibrary.from_expert_library(
    expert_lib,
    hf_repo_id,
    force=True,
    upload_aux_data=True,
)

In [ ]:
import pickle
# To save
pickle.dump(svd, open("svd.pkl", "wb"))

### PLayground